In [17]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import json
from bertopic import BERTopic
from hdbscan import HDBSCAN
import re

from pprint import pprint
import emoji

import os
from dotenv import load_dotenv

In [ ]:
load_dotenv(dotenv_path="./.env", override=True)
os.environ["AWS_BEARER_TOKEN_BEDROCK"] = os.getenv("Bedrock_API_Key")

AWS_BEARER_TOKEN_BEDROCK: ABSKQmVkcm9ja0FQSUtleS04Zmx0LWF0LTk5MjM4MjYxMjQ3NDprS3RrUng5Yk0wUisyTjJYVWgvUFRDMGUyVUxqaUg1cDhLdkp2TjhxRDlYcGFwRDRmT3VZaWIwbmYwWT0=


### jsonへの変換
pandas.DataFrameからto_json()メソッドを呼び出すと、デフォルトでは以下のようにJSON形式の文字列（str型）に変換される。<br>
全角文字（日本語）などのUnicodeエスケープ: 引数force_ascii<br>
https://note.nkmk.me/python-pandas-to-json/<br>

jsonのデータ出力を見やすくするには、pprintを使う<br>
https://zenn.dev/aew2sbee/articles/python-pprint

### envファイルによるAPIキーの取り扱い
load_dotenvにて、os.getenv("Bedrock_API_Key")によってAPIキーを取得する<br>
https://zenn.dev/querypie/articles/91aee7e3e51373

In [ ]:
def remove_emoji(text):
    return emoji.replace_emoji(text,replace='')

json_open = open("nogizaka_tweets.json","r",encoding="utf-8")
json_open = json.load(json_open)

id = []
text = []
viewCount = []
likeCount = []
url = []

for i in range(len(json_open)):
    id.append(json_open[i]["id"])
    text.append(json_open[i]["text"])
    viewCount.append(json_open[i]["viewCount"])
    likeCount.append(json_open[i]["likeCount"])
    url.append(json_open[i]["url"])

df = pd.DataFrame({
    "id":id,
    "text":text,
    "viewCount":viewCount,
    "likeCount":likeCount,
    "url":url})

df["text"] = df["text"].str.replace("\n","")
pattern = r'https?://[^\s]+'
df["text"] = df["text"].apply(lambda x:re.sub(pattern,"",x))

df["text"] = df["text"].apply(lambda x: remove_emoji(x))

df_json = df.to_json(orient='records',force_ascii=False)
#pprint(df_json)

('[{"id":"1970330163209789748","text":"＼絶賛発売中／#GoodsPress '
 '10月号連載一周年を記念してリバーシブル表紙に登場いただいた乃木坂46・冨里奈央さん！今回は、その表紙に使用したお写真をお届けします '
 '本編は10月号をチェック！  #グッズプレス ＃乃木坂46 ＃冨里奈央 '
 '＃NAOGRAPHIC","viewCount":50307,"likeCount":2305,"url":"https:\\/\\/x.com\\/GoodsPress\\/status\\/1970330163209789748"},{"id":"1967592475046097140","text":"色んな想いの人がいると思うけど、自分はアイドルが好きなんじゃなくて乃木坂46が大好き '
 '","viewCount":243350,"likeCount":4181,"url":"https:\\/\\/x.com\\/ikuma_nogi\\/status\\/1967592475046097140"},{"id":"1969265264924532742","text":"乃木坂46×So-netコラボキャンペーンサイトでは、CM撮影時のオフショットやメンバーからの裏話なども公開中こちらからご覧ください！ '
 '","viewCount":205970,"likeCount":6042,"url":"https:\\/\\/x.com\\/Sonet_fukuonsei\\/status\\/1969265264924532742"},{"id":"1968646188762316907","text":"次回の '
 '#乃木坂あそぶだけ は...NOGIZAKA ROOM SHARE ** '
 '┈┈┈┈┈┈┈**乃木坂46が、ひとつ屋根の下で“何をしてもいい”自由な時間を与えられたら？今回は5期生11名が登場 '
 '近日公開お楽しみに！#乃木坂46#のぎ動画 '
 '","viewCount":568518,"likeCount":6838,"url":"https:\\/\\/x.com\\/nogidoga46\\/status\\/1968646188762316907"},{"id":"1970

### まずは基本のクエリ
参考1：#基本のコード呼び出し：https://docs.aws.amazon.com/ja_jp/bedrock/latest/userguide/inference-invoke.html
<br>
参考2:#anthropicモデルの使い方：https://docs.aws.amazon.com/ja_jp/bedrock/latest/userguide/model-parameters-anthropic-claude-messages.html

### Langchainを利用<br>
参考：https://qiita.com/naito_kc/items/d3f6c390ba6238a2f66d<br>

#### langchainのPromptTemplateの使い方
input_variablesとtemplateで指定。LLMに渡す入力をテンプレート化する仕組み<br>
参考：https://qiita.com/kotai2003/items/7b924f47220ddadae497

#### json形式はプロンプトでも十分対応可能

参考：https://qiita.com/sekappy_official/items/f72c6c16c81f992f1bc8

In [36]:
import boto3
import json
from botocore.exceptions import ClientError

from langchain_core.prompts import PromptTemplate

client = boto3.client(
    "bedrock-runtime",
    region_name="us-east-1"
)

prompt = PromptTemplate(
    input_variables=["tweet_data"],
    template = """
    あなたは乃木坂46に詳しいニュース編集者です。
    以下は、X（旧Twitter）上で乃木坂46に関する最近のツイートデータです。

    「Xで話題になっている乃木坂46のトピック」として複数に分類してください。
    また各トピックに、日本語で簡潔なタイトルを1つ作成してください。そのうえでサンプルツイートも2つずつ挙げてください。

    制約条件：
    ・20文字以内
    ・具体的で、内容が一目で分かる
    ・感想ではなく事実ベース
    ・記号や絵文字は使わない

    出力方法：
    以下のキーを持つJSON形式でフォーマットしてください。JSONの前後に余計な説明文章は不要です。
    ・title // タイトル
    ・tweet_count // ツイート数
    ・total_pv // 総PV数
    ・total_favorites // 総お気に入り数
    ・sample_tweets // ツイート事例（配列で2つ）

    ツイートデータ：
    {tweet_data}
    """)
prompt = prompt.format(tweet_data=df_json)

body = json.dumps({
    "messages":[
        {"role":"user","content":prompt}
    ],
    "max_tokens":1024,
    "temperature":0.5,
    "anthropic_version":"bedrock-2023-05-31"
    
})

model_id = "anthropic.claude-3-5-sonnet-20240620-v1:0"

response = client.invoke_model(
    modelId=model_id,
    body=body
)

response_body = json.loads(response.get("body").read())
answer = response_body["content"][0]["text"]
print(answer)


{
    "topics": [
        {
            "title": "久保史緒里の卒業発表",
            "tweet_count": 12,
            "total_pv": 15164633,
            "total_favorites": 134253,
            "sample_tweets": [
                {
                    "text": "【ニュース更新】 久保史緒里 卒業のお知らせ ",
                    "viewCount": 9188028,
                    "likeCount": 74120
                },
                {
                    "text": "乃木坂46久保史緒里、卒業発表11月26・27日に横アリで卒コンコメント「今を逃してはならないと思いました」",
                    "viewCount": 3092847,
                    "likeCount": 21112
                }
            ]
        },
        {
            "title": "新曲Same numbersリリース",
            "tweet_count": 3,
            "total_pv": 16106,
            "total_favorites": 57,
            "sample_tweets": [
                {
                    "text": "乃木坂46 『Same numbers』リアミ３部 賀喜 遥香５年ぶりに間近で見る、かっきーは大人の女性になってました！！久しぶりに会えて良かった！！#賀喜遥香#ミーグリ#リアミ",
                    "viewCount": 8365,
                    "likeCount": 31
       

### EmbeddingをしたうえでLLMに読み込ませる →　ダメでした
最もEmbeddingの品質が良かった「paraphrase-multilingual-MiniLM-L12-v2」を使ってみる

In [28]:
from bertopic import BERTopic
from hdbscan import HDBSCAN

df_topic = df.copy()

# BERTopic の学習
hdbscan_model = HDBSCAN(min_cluster_size=5, min_samples=15, metric='euclidean', cluster_selection_method='eom')
model = BERTopic(embedding_model="paraphrase-multilingual-MiniLM-L12-v2",
                hdbscan_model=hdbscan_model)
topics, probs = model.fit_transform(df_topic["text"])

# df にトピック情報を追加
df_topic["topic"] = topics
df_topic["probability"] = probs  # 各文がクラスタに属する確率

# トピックごとの代表語を付与
topic_info = model.get_topic_info()
topic_repr = {row["Topic"]: row["Name"] for _, row in topic_info.iterrows()}
df_topic["topic_repr"] = df_topic["topic"].map(topic_repr)

df_topic = df_topic[df_topic["topic"] != -1]  # ノイズを除去
df_topic = df_topic[["text","topic","probability","topic_repr"]]
df_topic_json = df_topic.to_json(orient='records',force_ascii=False)

df_topic.head()

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


,text,topic,probability,topic_repr
0,＼絶賛発売中‼️／#GoodsPress 10月号連載一周年を記念してリバーシブル表紙に登場...,1,0.876142,1_乃木坂46_リアルミーグリ_一ノ瀬美空_ムーンライト伝説
1,色んな想いの人がいると思うけど、自分はアイドルが好きなんじゃなくて乃木坂46が大好き,0,1.000000,0_乃木坂46_今日は何の日_久保史緒里_ブログ更新
2,乃木坂46×So-netコラボキャンペーンサイトでは、CM撮影時のオフショットやメンバーから...,0,1.000000,0_乃木坂46_今日は何の日_久保史緒里_ブログ更新
3,次回の #乃木坂あそぶだけ は...NOGIZAKA ROOM SHARE 🏠** ┈┈┈┈...,0,1.000000,0_乃木坂46_今日は何の日_久保史緒里_ブログ更新
4,【今日は何の日】9月23日の乃木坂46,0,1.000000,0_乃木坂46_今日は何の日_久保史緒里_ブログ更新


In [29]:
import boto3
import json
from botocore.exceptions import ClientError

from langchain_core.prompts import PromptTemplate

client = boto3.client(
    "bedrock-runtime",
    region_name="us-east-1"
)

prompt = PromptTemplate(
    input_variables=["tweet_data"],
    template = """
    あなたは乃木坂46に詳しいニュース編集者です。
    以下は、X（旧Twitter）上で乃木坂46に関する最近のツイートデータです。

    「Xで話題になっている乃木坂46のトピック」としてトピックを複数作ったので、タイトルを作成してください
    そのうえでそのタイトルを適切に表すツイートを2つずつ挙げてください。

    制約条件：
    ・すでにトピックは「topic」カラムとして付与されていますので、そのトピックは変えないで。
    ・probabilityは各文がクラスタに属する確率です。
    ・topic_reprはトピックごとの代表語です。
    ・20文字以内
    ・具体的で、内容が一目で分かる
    ・感想ではなく事実ベース
    ・記号や絵文字は使わない

    ツイートデータ：
    {tweet_data}
    """)
prompt = prompt.format(tweet_data=df_topic_json)

body = json.dumps({
    "messages":[
        {"role":"user","content":prompt}
    ],
    "max_tokens":1024,
    "temperature":0.5,
    "anthropic_version":"bedrock-2023-05-31"
    
})

model_id = "anthropic.claude-3-5-sonnet-20240620-v1:0"

response = client.invoke_model(
    modelId=model_id,
    body=body
)

response_body = json.loads(response.get("body").read())
answer = response_body["content"][0]["text"]
print(answer)


トピック0: 乃木坂46の日常・活動情報

タイトル: 「乃木坂46メンバーの活動報告」

ツイート例:
1. 【ブログ更新 久保史緒里】 風のような、9年間でした  

2. 本日9月23日(火)23:00から、BAYFM「乃木坂46 柴田柚菜のDreaming time」に#柴田柚菜 が出演します✨皆さま、ぜひお聴きください🎃👻 

トピック1: 乃木坂46のビジュアル・ファッション

タイトル: 「乃木坂46メンバーの衣装・メイク」

ツイート例:
1. 💜乃木坂46 一ノ瀬美空ちゃん愛用カラコン■ chu's meベイビーブラウン¥1705🔗  Ever Colorはじらいベイビー¥1694🔗  #一ノ瀬美空 #リアルミーグリ 可愛い子のカラコン知れるの嬉しい...💕

2. 💜乃木坂46 川﨑桜ちゃん着用■ FRAY I.Dクラッシュベロアレイヤードカットソーandクラッシュベロアドロストワイドパンツ¥34,100#乃木坂46 #川﨑桜 #sakutangram 💡↓↓購入はこちら↓↓💡🔗
